In [48]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

### Load Data

In [78]:
df = pd.read_csv("../Data/fashion3.csv")
df.head()

,Unnamed: 0.1,Unnamed: 0,item_id,category,waist,size,quality,cup size,hips,bra size,height_inches,length,fit,review_summary,review_text,polarity,reviewLen,wordCount
0,0,8490,159891,1,26.0,1,5.0,2.0,38.0,32.0,64.0,5.0,3,This dress. Is so amazing,This dress. Is so amazing. It just arrived tod...,0.392500,301,58
1,1,8493,159891,1,31.0,8,3.0,5.0,41.0,36.0,62.0,5.0,3,This dress looks great on,This dress looks great on. I'm a pretty curren...,0.186250,273,57
2,2,8521,160625,1,32.0,5,5.0,2.0,NaN,32.0,65.0,5.0,1,Used as my wedding dress,Used as my wedding dress and I LOVED it!PROS ...,0.172436,452,85
3,3,8523,160625,1,32.0,11,4.0,5.0,43.0,38.0,67.0,5.0,2,"I love this dress, it's b","I love this dress, it's beautiful ! Wore this ...",0.413839,581,125
4,4,8524,160625,1,28.0,7,5.0,3.0,32.0,34.0,65.0,5.0,3,I love this dress!!! I bo,I love this dress!!! I bought it specifically ...,0.400769,627,117


In [79]:
df.iloc[:,2:13]
col_lst = list(range(2,13))
col_lst.extend([15,17])
df = df.iloc[:,col_lst]
df_clean = df.dropna()

In [80]:
df_clean.head()

,item_id,category,waist,size,quality,cup size,hips,bra size,height_inches,length,fit,polarity,wordCount
0,159891,1,26.0,1,5.0,2.0,38.0,32.0,64.0,5.0,3,0.392500,58
1,159891,1,31.0,8,3.0,5.0,41.0,36.0,62.0,5.0,3,0.186250,57
3,160625,1,32.0,11,4.0,5.0,43.0,38.0,67.0,5.0,2,0.413839,125
4,160625,1,28.0,7,5.0,3.0,32.0,34.0,65.0,5.0,3,0.400769,117
7,160625,1,27.0,5,5.0,3.0,38.0,36.0,68.0,5.0,2,0.098889,42


# SVD

User's rating scores:

* `quality`, and `polarity`

Because both features have different scaling, we need to make sure that they have same scaling to combine them together

In [81]:
# Scaling range: [0,1]
# After scaling, take the average to get the overall score
col_lst = [4,11]
score = df_clean.iloc[:,col_lst].apply(lambda iterator: ((iterator - iterator.min())/(iterator.max() - iterator.min())).round(2))
score["avg"] = score.mean(axis=1)
score.head()

,quality,polarity,avg
0,1.00,0.69,0.845
1,0.50,0.59,0.545
3,0.75,0.70,0.725
4,1.00,0.70,0.850
7,1.00,0.54,0.770


In [82]:
 # Add user id and category
score.insert(0, "user_id", score.index + 1)
score.insert(1, "item_id", df_clean["category"])

In [84]:
# only keep the first three columns
score = score.iloc[:,0:3]
score.head()

,user_id,item_id,quality
0,1,1,1.00
1,2,1,0.50
3,4,1,0.75
4,5,1,1.00
7,8,1,1.00


In [86]:
score = score.pivot(index='user_id', columns='item_id', values='quality') 

In [87]:
score.head()

item_id,1,2,3,4
user_id,,,,
1,1.00,NaN,NaN,NaN
2,0.50,NaN,NaN,NaN
4,0.75,NaN,NaN,NaN
5,1.00,NaN,NaN,NaN
8,1.00,NaN,NaN,NaN


#### 